In [6]:
import pandas as pd
import numpy as np
import json
import requests
import random

import os

from tqdm import tqdm

### 0. Convert ALL_Amazon_Meta.JSON to small file chunks

In [ ]:
"""
This code need not be run again if the category of items aren't changed
This code might need to be modified a little bit for actua folder structure and location of files etc.
"""

meta_file = 'All_Amazon_Meta.json/All_Amazon_Meta.json' # The original Extracted JSON file
file = open(meta_file)
required_category = 'Clothing, Shoes & Jewelry'     # Category of objectd to be extracted
write_file_name = 'Clothing_metadata'
selected_item_lists = []
total_selected_items = 0
current_selected_items = 0
write_file_counter = 0
for counter, line in enumerate(file):
    item = json.loads(line)
    if item['category'][0] == required_category:
        selected_item_lists.append(item)
        current_selected_items += 1
        total_selected_items += 1

    # if current_selected_items>= 2:
    if current_selected_items>=100000:
        file_name = write_file_name+'_'+str(write_file_counter)+'.json'
        with open(file_name,"w", encoding="utf8") as writer_file:
            json.dump(selected_item_lists,writer_file)
        write_file_counter+=1
        current_selected_items = 0
        selected_item_lists = []

    # if counter%1 == 0:
    if counter%1000000==0:
        print("Total lines: {}  | Total selected items : {} | Current selected items : {} | Write file counter : {}".format(
            counter+1, total_selected_items, current_selected_items, write_file_counter
        ))


In [ ]:
# Write all the reamining category items for one last time
file_name = write_file_name+'_'+str(write_file_counter)+'.json'
with open(file_name,"w", encoding="utf8") as writer_file:
    json.dump(selected_item_lists,writer_file)
write_file_counter+=1
current_selected_items = 0
selected_item_lists = []

### 1. Read ASINS from 5 core data

In [62]:
file_name = 'Clothing_Shoes_and_Jewelry_5.json'

asins = []
with open(file_name,'r') as f:
    for counter, line in enumerate(f):
        # print(line)
        asins.append(json.loads(line)['asin'])
        if counter%1000000==0:
            print("Items read : {}".format(counter))

# Write random 10000 product IDs to file
random_products = random.sample(list(set(asins)), 20000)
with open('Products_asin.txt','w', encoding='utf8') as file:
    file.write(str(random_products))

Items read : 0
Items read : 1000000
Items read : 2000000
Items read : 3000000
Items read : 4000000
Items read : 5000000
Items read : 6000000
Items read : 7000000
Items read : 8000000
Items read : 9000000
Items read : 10000000
Items read : 11000000


### 2. Get Metadata from ALL_AMAZON_META.json

In [64]:
meta_folder_name = 'Clothing_metadata'

with open('Products_asin.txt','r') as file:
    asins = eval(file.read())

for file_name in tqdm(os.listdir(meta_folder_name)):
    meta_df = pd.read_json(os.path.join(meta_folder_name, file_name))
    overlap_df = meta_df.loc[meta_df['asin'].isin(asins)]
    try:
        agg_meta_df = pd.concat([agg_meta_df,overlap_df])
    except:
        agg_meta_df = overlap_df.copy()

final_meta_df = agg_meta_df[agg_meta_df['description'].astype(bool) & 
                               agg_meta_df['title'].notna() & 
                               agg_meta_df['image'].astype(bool) & 
                               agg_meta_df['feature'].notna() & 
                               agg_meta_df['also_buy'].astype(bool) & 
                               agg_meta_df['also_view'].astype(bool) & 
                               agg_meta_df['price'].astype(bool)]


# Write Metadata to a file
data_folder = 'Project_Data'
data_filename = 'Metadata.csv'
if not os.path.exists(data_folder):
    os.mkdir(data_folder)
final_meta_df.to_csv(os.path.join(data_folder, data_filename))
    


100%|██████████| 27/27 [05:45<00:00, 12.81s/it]


### 3. Collect all review data for these products

In [66]:
list(final_meta_df.asin)

['B000072US4',
 'B000074RL3',
 'B000072UMA',
 'B00007KQBV',
 'B000087VAL',
 'B0000891KM',
 'B000089V5B',
 'B00008IOI3',
 'B00008KZCG',
 'B000099Q4V',
 'B00009UW4R',
 'B0000DZJLM',
 'B0000TIISW',
 'B0000WL0NY',
 'B00018C95A',
 'B0001Y8YBW',
 'B0001YS56G',
 'B0001YS5IE',
 'B0001YSBUG',
 'B0002LDVQW',
 'B0002LI9UA',
 'B0002LT6RU',
 'B0002MD73I',
 'B0002MGB9K',
 'B0002OQCJW',
 'B0002PLI0Y',
 'B0002T6AD0',
 'B0002TOZ2S',
 'B0002TOZ1Y',
 'B0002TOTEM',
 'B0002Z1IHC',
 'B0002ZI6Y0',
 'B0003DKVG2',
 'B000072US4',
 'B000074RL3',
 'B000072UMA',
 'B00007KQBV',
 'B000087VAL',
 'B0000891KM',
 'B000089V5B',
 'B00008IOI3',
 'B00008KZCG',
 'B000099Q4V',
 'B00009UW4R',
 'B0000DZJLM',
 'B0000TIISW',
 'B0000WL0NY',
 'B00018C95A',
 'B0001Y8YBW',
 'B0001YS56G',
 'B0001YS5IE',
 'B0001YSBUG',
 'B0002LDVQW',
 'B0002LI9UA',
 'B0002LT6RU',
 'B0002MD73I',
 'B0002MGB9K',
 'B0002OQCJW',
 'B0002PLI0Y',
 'B0002T6AD0',
 'B0002TOZ2S',
 'B0002TOZ1Y',
 'B0002TOTEM',
 'B0002Z1IHC',
 'B0002ZI6Y0',
 'B0003DKVG2',
 'B00065FZ

In [68]:
file_name = 'Clothing_Shoes_and_Jewelry_5.json'

asins = list(final_meta_df.asin)
reviews = []
with open(file_name,'r') as f:
    for line in tqdm(f):
        review = json.loads(line)
        if review['asin'] in asins:
            reviews.append(review)

# Write reviews as a CVS file
review_df = pd.DataFrame(reviews)
review_df.to_csv('Project_Data/Review_data.csv')

11285464it [48:37, 3867.80it/s]


### 4. Create Image data

In [ ]:
dataset_fname = 'Project_Data/Metadata.csv'
dataset = pd.read_csv(dataset_fname, index_col=0)
IMAGE_FOLDER_LR = 'Project_Data/Image_Data/Low_Res'
IMAGE_FOLDER_HR = 'Project_Data/Image_Data/High_Res'

if not os.path.exists(IMAGE_FOLDER_LR):
    os.makedirs(IMAGE_FOLDER_LR)
    
if not os.path.exists(IMAGE_FOLDER_HR):
    os.makedirs(IMAGE_FOLDER_HR)


image_dictionary = {}
success_count = 0
failed_count = 0
data_na = 0

# starting_index = 120000
# ending_index = 150000


# for index, product in tqdm(data_df.iloc[starting_index:ending_index, :].iterrows(), total = data_df.shape[0], initial = starting_index):
for index, product in tqdm(dataset.iterrows(), total = dataset.shape[0]):
        asin = product['asin']
        hr_urls = product['image']
        sucessful_extractions = []

        if hr_urls is np.nan:
            data_na +=1
            continue

        for ind, url in enumerate(eval(hr_urls)):
            try:
                extension = url.split('.')[-1]
                r = requests.get(url, allow_redirects=True)
                image_save_fname = asin + '_' + str(ind) +'.' + extension
                open(os.path.join(IMAGE_FOLDER_HR,image_save_fname), 'wb').write(r.content)
                sucessful_extractions.append(image_save_fname)
                success_count += 1
            except:
                failed_count += 1

            image_dictionary[asin] = sucessful_extractions


In [59]:
sample_dataset = final_meta_df[final_meta_df['description'].astype(bool) & 
                               final_meta_df['title'].notna() & 
                               final_meta_df['image'].astype(bool) & 
                               final_meta_df['feature'].notna() & 
                               final_meta_df['also_buy'].astype(bool) & 
                               final_meta_df['also_view'].astype(bool) & 
                               final_meta_df['price'].astype(bool)]


In [60]:
sample_dataset.shape

(3303, 18)

In [61]:
sample_dataset.head(5)

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
342,"[Clothing, Shoes & Jewelry, Women, Clothing, L...",,[Molded cups are perfect to wear under light c...,"class=""a-normal a-align-center a-spacing-smal...",Amoena Women's Mona Molded Wire-Free Bra,"[B017A8D1VE, B00X18EDCO, B00X18EMK2, B00X18END...",[https://images-na.ssl-images-amazon.com/image...,,Amoena,"[79% Nylon, 21% Spandex, Imported, Hand Wash, ...","333,937 in Clothing, Shoes & Jewelry (","[B00X18EN4W, B017A8D1VE, B00X18EDCO, B00X18END...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$8.52 - $90.46,B001ACT45E
1059,"[Clothing, Shoes & Jewelry, Novelty & More, Cl...",,[<b>All-Season Jerga</b><br> Dont leave the ho...,"class=""a-normal a-align-center a-spacing-smal...",Cleverbrand Unisex Mexican Jerga Hoodie (view ...,"[B017E4IJAM, B0756HXGHR, B00O95OY7C, B074WDVSY...",[https://images-na.ssl-images-amazon.com/image...,,Cleverbrand,"[50% cotton, 50% acrylic fibers, Features fron...","35,240 in Clothing, Shoes & Jewelry (","[B07CML6KB4, B0756HXGHR, B074WDVSY7, B00O95OY7...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$14.71 - $29.99,B001AIKKO2
2869,"[Clothing, Shoes & Jewelry, Men, Shoes, Sandals]",,"[Crocs, Inc. is a world leader in innovative c...","class=""a-normal a-align-center a-spacing-smal...",Crocs Women's Cleo Sandal,"[B01HP7LLBC, B072M82DQP, B00V822V8I, B01N9IF00...",[https://images-na.ssl-images-amazon.com/image...,,,"[100% Synthetic, Imported, Synthetic sole, Hee...","18,028 in Clothing, Shoes & Jewelry (","[B00V822V8I, B01HP7LLBC, B072M82DQP, B01N9IF00...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$19.15 - $279.08,B001AZJRCG
3935,"[Clothing, Shoes & Jewelry, Women, Shoes, Athl...",,[Fluid Tap Shoe Enhances Flexibility in the St...,"class=""a-normal a-align-center a-spacing-smal...",Capezio Women's CG17 Fluid Tap Shoe,"[B002CO2WSK, B002CQTY84, B002CQTZBU, B0041HYZR...",[https://images-na.ssl-images-amazon.com/image...,,,"[100% Synthetic, Leather and Synthetic sole, T...","13,042 in Clothing, Shoes & Jewelry (","[B07CK82BF2, B0041HYZR2, B07796S6BV, B07C9XDL8...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$31.32 - $66.45,B001B82ZDA
4181,"[Clothing, Shoes & Jewelry, Men, Accessories, ...",,[Maui Jim Sunglasses feature PolarizedPlus 2 t...,,Maui Jim Womens Makaha Sunglasses (405) Plasti...,"[B000MVNJS2, B00JLPGXFG, B072DTL4FM, B07MDDJ5S...",[https://images-na.ssl-images-amazon.com/image...,,Maui Jim,"[Acetate, Imported, Plastic frame, Plastic len...",[],"[B000MVNJS2, B002FU6UES, B07MDDJ5SF, B002FU6UE...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$229.00,B001BAD6Z4


In [28]:
final_meta_df.sample(5)

description
title
image
feature
also_buy
also_view
price
similar_item

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
77225,"[Clothing, Shoes & Jewelry, Boys]",,[Explore various terrains with the casual look...,"class=""a-normal a-align-center a-spacing-smal...",OLUKAI Kids Mens Nohea Moku (Toddler/Little Ki...,"[B00TDLHYDS, B019EA7JSC, B00XBL5B5O, B016SI24L...",[https://images-na.ssl-images-amazon.com/image...,,OLUKAI,"[Synthetic-and-mesh, Imported, Rubber sole, Ex...","557,755 in Clothing, Shoes & Jewelry (","[B019EA7JSC, B00TDLHYDS, B071LJ7297, B00XBL5B5...",{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$54.95 - $55.00,B01HEBA30E
82889,"[Clothing, Shoes & Jewelry, Women, Jewelry, Bo...",,[],,Belly Button Ring Reverse Tribal W/cz Ho800 14...,[B072XCDP91],[],,Body Accentz,[],"425,302 in Beauty & Personal Care (",[B01EHQVMPE],"{'Shipping Weight:': '0.2 ounces (', 'ASIN: ':...",All Beauty,,,$7.99,B008BMP92I
11572,"[Clothing, Shoes & Jewelry, Girls, Shoes, Athl...",,[Let her shimmer and shine in comfort with the...,,Skechers Skech Flex II Glitzy Glamour Girls Sl...,[],[https://images-na.ssl-images-amazon.com/image...,,,"[fabric, Rubber sole, Soft synthetic and mesh ...","2,790,893 in Clothing, Shoes & Jewelry (",[],{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",,B00GOZTTMA
96872,"[Clothing, Shoes & Jewelry, Women, Clothing]",,"[CLIMALITE soft, lightweight fabric for superi...","class=""a-normal a-align-center a-spacing-smal...",adidas Women's Sereno 11 Basic Pant,[],[https://images-na.ssl-images-amazon.com/image...,,adidas,"[100% polyester interlock., Imported, CLIMALIT...","339,733 in Sports & Outdoors (","[B01HO55N14, B010DK9C8W, B00Q6FKMDI, B077TJZWJ...","{'Shipping Weight:': '1 pounds (', 'Domestic S...",Sports & Outdoors,,,$40.00,B006YTX9KO
96627,"[Clothing, Shoes & Jewelry, Women, Clothing, T...",,[Women Striped Long Sleeve Loose Blouse Casual...,,Tnewold Women Striped Long Sleeve Loose Blouse...,[B01B60MV7M],[https://images-na.ssl-images-amazon.com/image...,,Tenworld,"[100% Polyester, Material: Polyester, perfect ...","1,821,304 in Clothing, Shoes & Jewelry (",[],{},"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",,B01D4VJSVS
